<a href="https://colab.research.google.com/github/pgordin/GraphsSN2024_1/blob/main/Graphs3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Package import.

In [36]:
import numpy as np
from random import random, seed
from copy import deepcopy

## Graph functions

In [2]:
def print_matrix(vertices, matrix):
  """
  Printing a graph given by adjacency matrix
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i, j]:
        print(" ", vv[j], end="")
    print("")

def print_graph(graph):
  """
  Printing of a graph (given as a dictionary/neighbouring list)
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")

## Making and modyfing graphs

In [3]:
def add_vertex(graph, vertex):
  """
  Add a new vertex to an existing graph
  """
  if vertex not in graph:
    graph[vertex] = []

def add_arc(graph, arc):
  """
  Given pair of vertices (arc variable) add an arc to an existing graph
  We consider simple, directed graphs.
  """
  u, v = arc
  add_vertex(graph, u)
  add_vertex(graph, v)
  if v not in graph[u]:
    graph[u].append(v)

def add_edge(graph, edge):
  """
  Given pair of vertices (edge variable) add an edge to existing graph.
  We consider simple, undirected graphs, as symmetric digraphs without loops.
  """
  u, v = edge
  add_vertex(graph, u)
  add_vertex(graph, v)
  if u == v:
    raise ValueError("Loops are not allowed!")
  if v not in graph[u]:
    graph[u].append(v)
  if u not in graph[v]:
    graph[v].append(u)


## Reading and writing graphs from/to files

In [4]:
def graph_from_edges(filename, directed = 0):
  """
  Read the graph from file, that in each line contains either
  the description of a vertex (one word) or
  the description of an edge/arc (at least 2 words).
  The resulting graph is returned as a neighbourhood list.
  Variable "filename" contains the whole path to the file.
  """
  graph = {}
  file = open(filename, "r")            # open the file to read
  for line in file:                     # for each line of the file
    words = line.strip().split()        # splits the line into words
    if len(words) == 1:                 # one word - vertex description
      add_vertex(graph, words[0])
    elif len(words) >= 2:               # at least two words, first two are the edge description
      if directed:
        add_arc(graph, (words[0], words[1]))
      else:
        add_edge(graph, (words[0], words[1]))
  file.close()
  return graph


def graph_to_neighbourlist(graph, filename):
  """
  Writes a graph (given as a disctionary/neighbouring list) to
  a textfile (as a neighbouring list).
  """
  file = open(filename, "w")    # open textfile for writing
  for v in graph:
    neigh_list = f"{v}:"
    for u in graph[v]:
      neigh_list = neigh_list + f" {u}"
    neigh_list = neigh_list + "\n"
    file.write(neigh_list)
  file.close()

In [42]:
def Prufer(tree):
  """
  Produces the Prufer code of a tree. It is necessary to have a tree as an input (it is not checked).
  Result is given as a string.
  """
  tr = deepcopy(tree)       # copy of a tree
  code = ""
  for i in range(len(tree)-2):
    for x in sorted(tr):
      if len(tr[x]) == 1:     # least leaf
        break
    v = tr[x][0]            # the only neighbour of x
    code = code + f"{v} "
    tr[v].remove(x)         # remove x from neighbours of v
    tr.pop(x)               # remove x from a tree
  return code.strip()


def TreeFromPrufer(code):
  """
  Creating a tree from a Prufer code
  """
  clist = [int(x) for x in code.split()]  # code as a list of numbers
  n = len(clist) + 2                  # number of vertices
  vert = [v for v in range(1, n+1)]   # list of numbers 1 .. n
  tree = {}
  for v in vert:
    add_vertex(tree, v)
  for i in range(n-2):
    for x in vert:
      if x not in clist:    # x - least leaf
        break
    v = clist.pop(0)    # remove the first element from the code - neighbour of x
    add_edge(tree, (x, v))
    vert.remove(x)
  add_edge(tree, (vert[0], vert[1]))
  return tree

## Use of code

### Part 3 (Prufer codes)

In [22]:
%%writefile tree1.txt
1 3
3 5
2 5
4 5
6 5

Writing tree1.txt


In [38]:
tree1 = graph_from_edges("tree1.txt")

In [39]:
print_graph(tree1)

1 :  3
3 :  1  5
5 :  3  2  4  6
2 :  5
4 :  5
6 :  5


In [40]:
print(Prufer(tree1))

3 5 5 5


In [41]:
print_graph(tree1)

1 :  3
3 :  1  5
5 :  3  2  4  6
2 :  5
4 :  5
6 :  5


In [43]:
print_graph(TreeFromPrufer("3 5 5 5"))

1 :  3
2 :  5
3 :  1  5
4 :  5
5 :  2  3  4  6
6 :  5


In [45]:
print_graph(TreeFromPrufer("5 4 3 1"))

1 :  3  6
2 :  5
3 :  4  1
4 :  5  3
5 :  2  4
6 :  1


In [46]:
print_graph(TreeFromPrufer("1 1 1 2 3"))

1 :  4  5  6  2
2 :  1  3
3 :  2  7
4 :  1
5 :  1
6 :  1
7 :  3


### Part2

In [21]:
%%writefile edges.txt
a b
b c
b d
d
d c
e
f


Overwriting edges.txt


In [6]:
%cat edges.txt

a b
b c
b d
d
d c
e
f


In [7]:
graph1 = graph_from_edges("edges.txt")
print_graph(graph1)

a :  b
b :  a  c  d
c :  b  d
d :  b  c
e :
f :


In [8]:
graph_to_neighbourlist(graph1, "neighbourhood.txt")

In [9]:
%cat "neighbourhood.txt"

a: b
b: a c d
c: b d
d: b c
e:
f:


In [10]:
!wget https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt

--2024-10-24 13:20:18--  https://github.com/pgordin/GraphsSN2024_1/raw/main/weighted0.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt [following]
--2024-10-24 13:20:19--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt’

weighted0.txt       100%[===================>]      99  --.-KB/s    in 0s      

2024-10-24 13:20:19 (4.52 MB/s) - ‘weighted0.txt’ saved [99/99]



In [11]:
graph2 = graph_from_edges("weighted0.txt")
print_graph(graph2)

A :  B  E
B :  A  C  D
E :  A  D  F  H
C :  B  D  F  G
D :  B  C  E  F
F :  C  D  E  G  H
G :  C  F  H  I
H :  E  F  G
I :  G


In [12]:
!wget https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/refs/heads/main/weighted0.txt

--2024-10-24 13:20:19--  https://raw.githubusercontent.com/pgordin/GraphsSN2024_1/refs/heads/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99 [text/plain]
Saving to: ‘weighted0.txt.1’

weighted0.txt.1     100%[===================>]      99  --.-KB/s    in 0s      

2024-10-24 13:20:19 (1.03 MB/s) - ‘weighted0.txt.1’ saved [99/99]



### Part1

In [13]:
vertices = ["a", "b", "c", "d"]
matrix = np.array([[0,1,0,0],[1,0,1,0],[0,1,0,1],[0,0,0,1]])
print(vertices)
print(matrix)
print("---------------------------")
print_matrix(vertices, matrix)
print("---------------------------")
print_matrix(None,matrix)

['a', 'b', 'c', 'd']
[[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]]
---------------------------
a :  b
b :  a  c
c :  b  d
d :  d
---------------------------
1 :  2
2 :  1  3
3 :  2  4
4 :  4


In [14]:
graph = {
  "a": ["b"],
  "b": ["a", "c"],
  "c": ["b", "d"],
  "d": ["c"]
}
print(graph)
print("---------------------------")
print_graph(graph)

{'a': ['b'], 'b': ['a', 'c'], 'c': ['b', 'd'], 'd': ['c']}
---------------------------
a :  b
b :  a  c
c :  b  d
d :  c


In [15]:
add_vertex(graph, "e")
print_graph(graph)

a :  b
b :  a  c
c :  b  d
d :  c
e :


In [16]:
add_edge(graph, ["e", "f"])
print_graph(graph)

a :  b
b :  a  c
c :  b  d
d :  c
e :  f
f :  e


In [17]:
add_arc(graph, ["e", "a"]) # breaking the symmetry
print_graph(graph)

a :  b
b :  a  c
c :  b  d
d :  c
e :  f  a
f :  e


In [18]:
add_edge(graph, ["a", "e"]) # restoring the symmetry
print_graph(graph)

a :  b  e
b :  a  c
c :  b  d
d :  c
e :  f  a
f :  e


In [19]:
add_edge(graph, ["e", "f"]) # do nothing, an edge already exists
print_graph(graph)

a :  b  e
b :  a  c
c :  b  d
d :  c
e :  f  a
f :  e


In [20]:
add_edge(graph, ["e", "e"]) # an error

ValueError: Loops are not allowed!

In [ ]:
add_arc(graph, ["e", "e"]) # OK - loops are allowed in digraphs
print_graph(graph)

## Random graphs generator in the $G(n,p)$ model.

In [ ]:
# for repeatness
seed(2024)

In [ ]:
#draw a graph G(10, 1/3)
n = 10
p = 1/3
random_graph = {}
for i in range(1, n+1):
  add_vertex(random_graph, i)
  for j in range(1, i):
    if random() < p:
      add_edge(random_graph, [i, j])

print_graph(random_graph)